In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras.layers import LSTM, Dense, Bidirectional

In [2]:
!!curl -O http://www.manythings.org/anki/fra-eng.zip

['  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current',
 '                                 Dload  Upload   Total   Spent    Left  Speed',
 '',
 '  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0',
 '100     5    0     5    0     0      5      0 --:--:-- --:--:-- --:--:--     5',
 '100     5    0     5    0     0      5      0 --:--:-- --:--:-- --:--:--     5']

In [10]:
data_path = "fra.txt"

In [11]:
batch_size = 64
epochs = 100
latent_dim = 256
num_samples = 10000

In [12]:
## Input and data processing

In [13]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split("\t")
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 70
Number of unique output tokens: 93
Max sequence length for inputs: 14
Max sequence length for outputs: 59


In [14]:
##Give indexs to charcters
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

In [15]:
##INtialise the data #D encodr_input,decoder_input and decoder_target
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

In [16]:
##One Hot Representation of the sentences
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

In [17]:
# Define an input sequence and process it.
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)


In [18]:
model.compile(
    optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.fit([encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)
# Save model
model.save("french_to_eng")

Epoch 1/100
125/125 [==============================] - 18s 130ms/step - loss: 1.3197 - accuracy: 0.7277 - val_loss: 1.0404 - val_accuracy: 0.7064
Epoch 2/100
125/125 [==============================] - 16s 129ms/step - loss: 0.8674 - accuracy: 0.7614 - val_loss: 0.8843 - val_accuracy: 0.7578
Epoch 3/100
125/125 [==============================] - 18s 143ms/step - loss: 0.7268 - accuracy: 0.8027 - val_loss: 0.7492 - val_accuracy: 0.7893
Epoch 4/100
125/125 [==============================] - 17s 134ms/step - loss: 0.6254 - accuracy: 0.8240 - val_loss: 0.6624 - val_accuracy: 0.8134
Epoch 5/100
125/125 [==============================] - 17s 134ms/step - loss: 0.5680 - accuracy: 0.8372 - val_loss: 0.6168 - val_accuracy: 0.8226
Epoch 6/100
125/125 [==============================] - 17s 137ms/step - loss: 0.5302 - accuracy: 0.8468 - val_loss: 0.5847 - val_accuracy: 0.8304
Epoch 7/100
125/125 [==============================] - 17s 139ms/step - loss: 0.5019 - accuracy: 0.8548 - val_loss: 0.5658 -

Epoch 57/100
125/125 [==============================] - 18s 142ms/step - loss: 0.1071 - accuracy: 0.9665 - val_loss: 0.5299 - val_accuracy: 0.8763
Epoch 58/100
125/125 [==============================] - 18s 141ms/step - loss: 0.1032 - accuracy: 0.9676 - val_loss: 0.5373 - val_accuracy: 0.8751
Epoch 59/100
125/125 [==============================] - 18s 142ms/step - loss: 0.0994 - accuracy: 0.9689 - val_loss: 0.5367 - val_accuracy: 0.8759
Epoch 60/100
125/125 [==============================] - 16s 130ms/step - loss: 0.0967 - accuracy: 0.9695 - val_loss: 0.5494 - val_accuracy: 0.8744
Epoch 61/100
125/125 [==============================] - 16s 132ms/step - loss: 0.0947 - accuracy: 0.9700 - val_loss: 0.5497 - val_accuracy: 0.8746
Epoch 62/100
125/125 [==============================] - 18s 145ms/step - loss: 0.0924 - accuracy: 0.9706 - val_loss: 0.5557 - val_accuracy: 0.8742
Epoch 63/100
125/125 [==============================] - 18s 140ms/step - loss: 0.0884 - accuracy: 0.9717 - val_loss: 0

INFO:tensorflow:Assets written to: french_to_eng\assets


INFO:tensorflow:Assets written to: french_to_eng\assets


In [19]:
# Define sampling models
# Restore the model and construct the encoder and decoder.
model = keras.models.load_model("french_to_eng")

encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = keras.Input(shape=(latent_dim,), name="input_3")
decoder_state_input_c = keras.Input(shape=(latent_dim,), name="input_4")
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence

In [20]:
for seq_index in range(20):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)

1/1 [==============================] - 0s 21ms/step
-
Input sentence: Go.
Decoded sentence: Va !

1/1 [==============================] - 0s 23ms/step
-
Input sentence: Go.
Decoded sentence: Va !

1/1 [==============================] - 0s 29ms/step
-
Input sentence: Go.
Decoded sentence: Va !

1/1 [==============================] - 0s 15ms/step
-
Input sentence: Go.
Decoded sentence: Va !

1/1 [==============================] - 0s 30ms/step
-
Input sentence: Hi.
Decoded sentence: Salut !

1/1 [==============================] - 0s 29ms/step
-
Input sentence: Hi.
Decoded sentence: Salut !

1/1 [==============================] - 0s 10ms/step
-
Input sentence: Run!
Decoded sentence: Fuyez !

1/1 [==============================] - 0s 7ms/step
-
Input sentence: Run!
Decoded sentence: Fuyez !

1/1 [==============================] - 0s 19ms/step
-
Input sentence: Run!
Decoded sentence: Fuyez !

1/1 [==============================] - 0s 15ms/step
-
Input sentence: Run!
Decoded sentence: Fuyez !


1/1 [==============================] - 0s 15ms/step
-
Input sentence: Run.
Decoded sentence: Fuyons !

1/1 [==============================] - 0s 16ms/step
-
Input sentence: Run.
Decoded sentence: Fuyons !

1/1 [==============================] - 0s 16ms/step
-
Input sentence: Run.
Decoded sentence: Fuyons !

1/1 [==============================] - 0s 15ms/step
-
Input sentence: Run.
Decoded sentence: Fuyons !

